In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers as kl
import random
from pickle import load

In [3]:
with open('winning_data.pkl', 'rb') as f:
    f.seek(0)
    winning_data = load(f)
with open('indextocard.pkl', 'rb') as f:
    f.seek(0)
    card_index = load(f)
with open('cardtoindex.pkl', 'rb') as f:
    f.seek(0)
    card_dict = load(f)

In [5]:
print(len(winning_data))

44373


In [6]:
def datasetup(draft):
    cards = 0
    alldrafted = np.zeros(343, np.int8)
    maindeck = np.zeros(343, np.int8)
    for key,value in draft.items():
        if key.startswith("main_"):
            cards = cards + int(value)
            card = key.split("main_")[1]
            maindeck[int(card_dict[card])] = value
        if key.startswith("draft_"):
            card = key.split("draft_")[1]
            alldrafted[int(card_dict[card])] = value
    return cards, alldrafted, maindeck

In [7]:
def getremovedcards(deck, draft):
    removedlist = []
    draftlist = []
    for index, value in enumerate(deck):
        for i in range(0, value):
            array = np.zeros(343, np.int8)
            array[index] = 1
            removedlist.append(array)
            draftlist.append(draft)
    removedlist = np.array(removedlist) 
    draftlist = np.array(draftlist) 
    return removedlist, draftlist

In [8]:
def removerandomcards(deck):
  copy_deck = np.zeros(343)
  for index, value in enumerate(deck):
    if value > 0 and random.random() > 0.25:
      copy_deck[index] = copy_deck[index] + random.randint(0, value)
  return copy_deck

In [9]:
def printcards(deck):
    for index, card in enumerate(deck):
        if card > 0:
            print(card_index[index] + ", " + str(card))

In [10]:
def specialround(deck, draft):
  copydeck = deck.copy()
  for index, value in enumerate(copydeck):
    if value < 0.5:
      copydeck[index] = 0
    else:
      copydeck[index] = min(np.ceil(value), draft[index])
  return copydeck

In [11]:
def totalcards(deck):
  x = 0
  for i in deck:
    x += i
  return x

In [12]:
def calcaccuracy(pred, main):
  diff = 0
  total = 0
  for index, value in enumerate(pred):
    if value != main[index]:
      diff += abs(value - main[index])
    total += main[index]
  return 1 - diff/total

In [13]:
def exporttodraft(deckexport):
  draft = np.zeros(343)
  maindeck = np.zeros(343)
  includemain = True
  for index, value in enumerate(deckexport):
    if value.startswith('Sideboard'):
      includemain = False
    split = value.split(" ", 1)
    if len(split) == 2:
      name = split[1].split("(", 1)
      card_name = name[0].strip()
      if card_name == "Mountain" or card_name == "Island" or card_name == "Plains" or card_name == "Swamp" or card_name == "Forest":
        continue
      cardindex = card_dict[card_name]
      draft[cardindex] = draft[cardindex] + int(split[0])
      if includemain:
        maindeck[cardindex] = maindeck[cardindex] + int(split[0])
  return draft, maindeck

In [14]:
cardcount, alldrafted, removedmain = datasetup(winning_data[1])
remo = removerandomcards(removedmain)
printcards(remo)
print("---------------")
printcards(removedmain)
print(removedmain.shape)
concat = np.concatenate((removedmain, remo))
print(concat.shape)

Closing Statement, 1.0
Dramatic Finale, 1.0
Extus, Oriq Overlord, 1.0
Hall of Oracles, 1.0
Hunt for Specimens, 1.0
Rise of Extus, 1.0
Silverquill Apprentice, 1.0
Silverquill Pledgemage, 1.0
Stonerise Spirit, 1.0
Tenured Inkcaster, 1.0
Unwilling Ingredient, 1.0
---------------
Closing Statement, 1
Combat Professor, 1
Dramatic Finale, 1
Extus, Oriq Overlord, 1
Eyetwitch, 1
Hall of Oracles, 1
Hunt for Specimens, 1
Owlin Shieldmage, 1
Rise of Extus, 3
Silverquill Apprentice, 1
Silverquill Campus, 1
Silverquill Pledgemage, 1
Star Pupil, 5
Stonerise Spirit, 2
Study Break, 1
Tenured Inkcaster, 2
Unwilling Ingredient, 1
(343,)
(686,)


In [15]:
npp = np.load('data/' + str(1) + '.npy')
print(npp[1][0])

FileNotFoundError: ignored

In [ ]:
class DataGenerator(keras.utils.Sequence):
    """Generates data for Keras"""

    def __init__(self, start, end, batch_size, shuffle=False):
        """Initialization"""
        self.data_indexes = np.arange(start, end)
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.data_indexes) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""
        # Generate indexes of the batch
        indexes = self.data_indexes[index*self.batch_size:(index+1)*self.batch_size]

        X = np.empty((self.batch_size, 686))
        y = np.empty((self.batch_size, 343))

        # Generate data
        for i, ID in enumerate(indexes):
            npp = np.load('data/' + str(ID) + '.npy')
            y[i] = npp[1][0]
            remo = removerandomcards(npp[1][0])
            concat = np.concatenate((npp[0][0], remo))
            X[i] = concat

        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle:
            np.random.shuffle(self.data_indexes)

In [ ]:
model = keras.Sequential()

model.add(kl.Dense(343, input_shape=(686, ), activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))

model = keras.models.load_model("MainDeckModel.h5")

print(model.summary(90))

metric = keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None)
model.compile(loss='mean_squared_error',
              optimizer='adam', metrics=[metric])

data_gen = DataGenerator(0, 38400, 8, shuffle=True)
valid_gen = DataGenerator(38400, 44373, 32)

my_callbacks = [
    keras.callbacks.EarlyStopping(patience=5),
    keras.callbacks.ModelCheckpoint(filepath="MainDeckModel.h5",
                                    save_best_only=True),
]

model.fit(data_gen, steps_per_epoch=4800, epochs=100, validation_data=valid_gen, validation_steps=186,
          callbacks=my_callbacks, shuffle=False, use_multiprocessing=True,
          workers=4)

Model: "sequential"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
dense (Dense)                           (None, 343)                         235641        
__________________________________________________________________________________________
dense_1 (Dense)                         (None, 343)                         117992        
__________________________________________________________________________________________
dense_2 (Dense)                         (None, 343)                         117992        
__________________________________________________________________________________________
dense_3 (Dense)                         (None, 343)                         117992        
__________________________________________________________________________________________
dense_4 (Dense)                         (None, 343)                   

In [ ]:
model = keras.models.load_model("MainDeckModel30-0218.h5")
valid_gen = DataGenerator(38400, 44373, 32)
result = model.predict(valid_gen)

In [ ]:
totalacc = 0
for index, value in enumerate(result):
  nppp = np.load('data/' + str(index + 38400) + '.npy')
  totalacc += calcaccuracy(specialround(value, nppp[0][0]), nppp[1][0])
print(totalacc/len(result))

0.5533019359393402


In [19]:
f = open("testdeck.txt", "r")
deckexport = f.readlines()
draft, maindeck = exporttodraft(deckexport)
realmodel = keras.models.load_model("MainDeckModelFinal.h5")
realtest = realmodel.predict(np.array([draft]))

In [20]:
preddeck = specialround(realtest[0], draft)
printcards(draft)
print("----- Predicted Deck: Cards " + str(totalcards(preddeck)))
printcards(preddeck)
print("----- Main Deck: Cards " + str(totalcards(maindeck)))
printcards(maindeck)
print(calcaccuracy(preddeck, maindeck))

Ageless Guardian, 2.0
Blood Age General, 2.0
Cogwork Archivist, 1.0
Curate, 1.0
Defend the Campus, 2.0
Defiant Strike, 1.0
Divine Gambit, 1.0
Dragon's Approach, 1.0
Dueling Coach, 2.0
Exhilarating Elocution, 1.0
Expel, 1.0
Fuming Effigy, 1.0
Golden Ratio, 1.0
Guiding Voice, 1.0
Heated Debate, 2.0
Introduction to Prophecy, 1.0
Lorehold Apprentice, 1.0
Lorehold Campus, 1.0
Lorehold Excavation, 1.0
Make Your Mark, 1.0
Pigment Storm, 3.0
Quintorius, Field Historian, 1.0
Rip Apart, 1.0
Show of Confidence, 1.0
Solve the Equation, 1.0
Soothsayer Adept, 1.0
Spined Karok, 1.0
Star Pupil, 1.0
Stonerise Spirit, 2.0
Storm-Kiln Artist, 1.0
Strategic Planning, 1.0
Study Break, 1.0
Team Pennant, 1.0
Thrilling Discovery, 1.0
Tome Shredder, 2.0
Waterfall Aerialist, 1.0
----- Predicted Deck: Cards 27.0
Ageless Guardian, 2.0
Blood Age General, 2.0
Defend the Campus, 2.0
Divine Gambit, 1.0
Dueling Coach, 2.0
Fuming Effigy, 1.0
Guiding Voice, 1.0
Heated Debate, 2.0
Lorehold Excavation, 1.0
Make Your Mark, 

In [21]:
model = keras.models.load_model("MainDeckModel30-0218.h5")
concat = np.concatenate((draft, maindeck))
testing = model.predict(np.array([concat]))
preddeck = specialround(testing[0], draft)
printcards(draft)
print("----- Predicted Deck: Cards " + str(totalcards(preddeck)))
printcards(preddeck)
print("----- Main Deck: Cards " + str(totalcards(maindeck)))
printcards(maindeck)
print(calcaccuracy(preddeck, maindeck))

Ageless Guardian, 2.0
Blood Age General, 2.0
Cogwork Archivist, 1.0
Curate, 1.0
Defend the Campus, 2.0
Defiant Strike, 1.0
Divine Gambit, 1.0
Dragon's Approach, 1.0
Dueling Coach, 2.0
Exhilarating Elocution, 1.0
Expel, 1.0
Fuming Effigy, 1.0
Golden Ratio, 1.0
Guiding Voice, 1.0
Heated Debate, 2.0
Introduction to Prophecy, 1.0
Lorehold Apprentice, 1.0
Lorehold Campus, 1.0
Lorehold Excavation, 1.0
Make Your Mark, 1.0
Pigment Storm, 3.0
Quintorius, Field Historian, 1.0
Rip Apart, 1.0
Show of Confidence, 1.0
Solve the Equation, 1.0
Soothsayer Adept, 1.0
Spined Karok, 1.0
Star Pupil, 1.0
Stonerise Spirit, 2.0
Storm-Kiln Artist, 1.0
Strategic Planning, 1.0
Study Break, 1.0
Team Pennant, 1.0
Thrilling Discovery, 1.0
Tome Shredder, 2.0
Waterfall Aerialist, 1.0
----- Predicted Deck: Cards 26.0
Ageless Guardian, 2.0
Blood Age General, 2.0
Cogwork Archivist, 1.0
Defend the Campus, 1.0
Divine Gambit, 1.0
Dueling Coach, 2.0
Fuming Effigy, 1.0
Guiding Voice, 1.0
Heated Debate, 2.0
Lorehold Apprentic

In [ ]:
npp = np.load('data/' + str(44004) + '.npy')
remo = np.zeros(343)
concat = np.concatenate((npp[0][0], remo))
testing = model.predict(np.array([concat]))

In [ ]:
printcards(remo)

In [ ]:
preddeck = specialround(testing[0], npp[0][0])
printcards(npp[0][0])
print(totalcards(preddeck))
printcards(preddeck)
printcards(npp[1][0])
print(calcaccuracy(preddeck, npp[1][0]))

Academic Dispute, 1
Aether Helix, 1
Arcane Subtraction, 1
Bury in Books, 4
Claim the Firstborn, 1
Crux of Fate, 1
Cultivate, 1
Elemental Masterpiece, 1
Eliminate, 1
Ephemerate, 1
Exhilarating Elocution, 1
Expel, 1
Exponential Growth, 1
Field Trip, 2
Furycalm Snarl, 1
Hall of Oracles, 1
Ingenious Mastery, 1
Inquisition of Kozilek, 1
Introduction to Prophecy, 1
Leyline Invocation, 1
Mortality Spear, 1
Prismari Pledgemage, 1
Professor Onyx, 1
Quandrix Apprentice, 1
Quandrix Campus, 1
Quandrix Cultivator, 1
Reckless Amplimancer, 1
Reflective Golem, 1
Relic Sloth, 1
Scurrid Colony, 1
Silverquill Command, 1
Snakeskin Veil, 1
Soothsayer Adept, 1
Sparring Regimen, 1
Square Up, 1
Star Pupil, 1
Storm-Kiln Artist, 1
Tempted by the Oriq, 1
Velomachus Lorehold, 1
Vortex Runner, 1
Waterfall Aerialist, 1
21.0
Academic Dispute, 1.0
Aether Helix, 1.0
Arcane Subtraction, 1.0
Bury in Books, 3.0
Cultivate, 1.0
Elemental Masterpiece, 1.0
Field Trip, 2.0
Leyline Invocation, 1.0
Prismari Pledgemage, 1.0
Quan